In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, \
  ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, matthews_corrcoef

In [2]:
data = pd.read_csv(r'..\..\10_fold_cross_validation\train_10folds.csv')
data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET,Kfold
0,0.335201,1.714286,-2.072068,5.975550,0.000000,12.132734,9.551078,0.000000,38.974594,43.638476,6,0,1,1.0,0
1,0.226791,2.000000,-2.362642,7.150190,5.948339,0.000000,9.967957,0.000000,10.440599,11.336786,6,0,0,1.0,1
2,0.211302,1.772727,-2.089721,7.912349,9.837253,29.297126,4.983979,9.837253,10.902925,24.265468,4,0,1,0.0,1
3,0.226898,1.869565,-2.357502,6.433493,5.948339,35.334614,15.284746,0.000000,11.126903,12.263211,5,0,0,0.0,0
4,0.158370,1.870968,-2.421374,7.991366,0.000000,23.362825,19.935914,6.420822,0.000000,18.460054,9,0,1,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.152613,2.000000,-2.415835,6.432407,0.000000,35.334614,9.967957,11.343745,11.257379,6.066367,6,0,0,1.0,3
2735,0.335203,1.700000,-2.001320,6.164147,0.000000,30.331835,4.983979,0.000000,10.902925,36.528679,2,0,1,0.0,6
2736,0.264178,1.833333,-2.414068,5.737009,5.948339,38.112943,9.551078,11.784535,11.126903,11.614772,7,0,0,1.0,8
2737,0.338995,1.815789,-2.065301,7.218862,5.907180,59.014740,24.544948,18.386966,43.634305,6.196844,9,0,2,0.0,5


In [3]:
def run(fold, data):
    # load the full training data with folds
    df = data
    # all columns are features except target and kfold columns
    features = [
        f for f in df.columns if f not in ("TARGET", "Kfold")
    ]
    # get training data using folds
    df_train = df[df.Kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = df[df.Kfold == fold].reset_index(drop=True)
    # get training data
    X_train = df_train[features].values
    # get validation data
    X_valid = df_valid[features].values
    # initialize Logistic Regression model
    model = DecisionTreeClassifier()
    model.fit(X_train, df_train.TARGET.values)
    valid_preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(df_valid.TARGET.values, valid_preds)
    y_pred = model.predict(X_valid)
    y_true = df_valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [4]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    filename = 'DecisionTree_' + str(fold_) + '.pkl'
    joblib.dump(model, filename)
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

Fold = 0, AUC = 0.9466189061680936, Accuracy = 0.9452554744525548,           Precision_1 = 0.9361702127659575, Precision_0 = 0.95          Recall_1 = 0.9072164948453608, Recall_0 = 0.9661016949152542, F1Score = 0.9214659685863875, kappa = 0.8794650712651768, MCC = 0.879720731841955
Fold = 1, AUC = 0.9096918865396937, Accuracy = 0.9197080291970803,           Precision_1 = 0.9120879120879121, Precision_0 = 0.9234972677595629          Recall_1 = 0.8556701030927835, Recall_0 = 0.9548022598870056, F1Score = 0.8829787234042552, kappa = 0.8219623131903834, MCC = 0.8229329834086583
Fold = 2, AUC = 0.9001106645698643, Accuracy = 0.9014598540145985,           Precision_1 = 0.8804347826086957, Precision_0 = 0.9120879120879121          Recall_1 = 0.8350515463917526, Recall_0 = 0.9378531073446328, F1Score = 0.8571428571428571, kappa = 0.7820142613000176, MCC = 0.7826522081504074
Fold = 3, AUC = 0.9179917292795153, Accuracy = 0.916058394160584,           Precision_1 = 0.8627450980392157, Precision_0

In [5]:
fold_metrics = pd.DataFrame(columns=['Accuracy','AUC','Precision_1','Precision_0','Recall_1','Recall_0','F1score','Kappa','MCC'])
fold_metrics['Accuracy'] = np.array(accuracies)
fold_metrics['AUC'] = np.array(aucs)
fold_metrics['Precision_1'] = np.array(precisions_1)
fold_metrics['Precision_0'] = np.array(precisions_0)
fold_metrics['Recall_1'] = np.array(recalls_1)
fold_metrics['Recall_0'] = np.array(recalls_0)
fold_metrics['F1score'] = np.array(f1scores)
fold_metrics['Kappa'] = np.array(kappas)
fold_metrics['MCC'] = np.array(MCCs)
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.945255,0.946619,0.936170,0.950000,0.907216,0.966102,0.921466,0.879465,0.879721
1,0.919708,0.909692,0.912088,0.923497,0.855670,0.954802,0.882979,0.821962,0.822933
2,0.901460,0.900111,0.880435,0.912088,0.835052,0.937853,0.857143,0.782014,0.782652
3,0.916058,0.917992,0.862745,0.947674,0.907216,0.920904,0.884422,0.818585,0.819222
4,0.930657,0.926350,0.890000,0.954023,0.917526,0.937853,0.903553,0.849442,0.849682
5,0.934307,0.937504,0.907216,0.949153,0.907216,0.949153,0.907216,0.856369,0.856369
6,0.919708,0.927893,0.878788,0.942857,0.896907,0.932203,0.887755,0.825265,0.825369
7,0.894161,0.890995,0.854167,0.915730,0.845361,0.920904,0.849741,0.768054,0.768079
8,0.919708,0.909410,0.893617,0.933333,0.875000,0.943820,0.884211,0.822768,0.822875
9,0.963370,0.961864,0.938776,0.977143,0.958333,0.966102,0.948454,0.920049,0.920167


In [6]:
fold_metrics.loc[10,:] = [np.mean(np.array(accuracies)), np.mean(np.array(aucs)), np.mean(np.array(precisions_1)),
                               np.mean(np.array(precisions_0)), np.mean(np.array(recalls_1)), np.mean(np.array(recalls_0)),
                            np.mean(np.array(f1scores)), np.mean(np.array(kappas)), np.mean(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.945255,0.946619,0.936170,0.950000,0.907216,0.966102,0.921466,0.879465,0.879721
1,0.919708,0.909692,0.912088,0.923497,0.855670,0.954802,0.882979,0.821962,0.822933
2,0.901460,0.900111,0.880435,0.912088,0.835052,0.937853,0.857143,0.782014,0.782652
3,0.916058,0.917992,0.862745,0.947674,0.907216,0.920904,0.884422,0.818585,0.819222
4,0.930657,0.926350,0.890000,0.954023,0.917526,0.937853,0.903553,0.849442,0.849682
5,0.934307,0.937504,0.907216,0.949153,0.907216,0.949153,0.907216,0.856369,0.856369
6,0.919708,0.927893,0.878788,0.942857,0.896907,0.932203,0.887755,0.825265,0.825369
7,0.894161,0.890995,0.854167,0.915730,0.845361,0.920904,0.849741,0.768054,0.768079
8,0.919708,0.909410,0.893617,0.933333,0.875000,0.943820,0.884211,0.822768,0.822875
9,0.963370,0.961864,0.938776,0.977143,0.958333,0.966102,0.948454,0.920049,0.920167


In [7]:
fold_metrics.loc[11,:] = [np.std(np.array(accuracies)), np.std(np.array(aucs)), np.std(np.array(precisions_1)),
                               np.std(np.array(precisions_0)), np.std(np.array(recalls_1)), np.std(np.array(recalls_0)),
                            np.std(np.array(f1scores)), np.std(np.array(kappas)), np.std(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.945255,0.946619,0.936170,0.950000,0.907216,0.966102,0.921466,0.879465,0.879721
1,0.919708,0.909692,0.912088,0.923497,0.855670,0.954802,0.882979,0.821962,0.822933
2,0.901460,0.900111,0.880435,0.912088,0.835052,0.937853,0.857143,0.782014,0.782652
3,0.916058,0.917992,0.862745,0.947674,0.907216,0.920904,0.884422,0.818585,0.819222
4,0.930657,0.926350,0.890000,0.954023,0.917526,0.937853,0.903553,0.849442,0.849682
5,0.934307,0.937504,0.907216,0.949153,0.907216,0.949153,0.907216,0.856369,0.856369
6,0.919708,0.927893,0.878788,0.942857,0.896907,0.932203,0.887755,0.825265,0.825369
7,0.894161,0.890995,0.854167,0.915730,0.845361,0.920904,0.849741,0.768054,0.768079
8,0.919708,0.909410,0.893617,0.933333,0.875000,0.943820,0.884211,0.822768,0.822875
9,0.963370,0.961864,0.938776,0.977143,0.958333,0.966102,0.948454,0.920049,0.920167


In [8]:
fold_metrics.index = ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7','Fold_8','Fold_9','Mean','Std']
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
Fold_0,0.945255,0.946619,0.936170,0.950000,0.907216,0.966102,0.921466,0.879465,0.879721
Fold_1,0.919708,0.909692,0.912088,0.923497,0.855670,0.954802,0.882979,0.821962,0.822933
Fold_2,0.901460,0.900111,0.880435,0.912088,0.835052,0.937853,0.857143,0.782014,0.782652
Fold_3,0.916058,0.917992,0.862745,0.947674,0.907216,0.920904,0.884422,0.818585,0.819222
Fold_4,0.930657,0.926350,0.890000,0.954023,0.917526,0.937853,0.903553,0.849442,0.849682
Fold_5,0.934307,0.937504,0.907216,0.949153,0.907216,0.949153,0.907216,0.856369,0.856369
Fold_6,0.919708,0.927893,0.878788,0.942857,0.896907,0.932203,0.887755,0.825265,0.825369
Fold_7,0.894161,0.890995,0.854167,0.915730,0.845361,0.920904,0.849741,0.768054,0.768079
Fold_8,0.919708,0.909410,0.893617,0.933333,0.875000,0.943820,0.884211,0.822768,0.822875
Fold_9,0.963370,0.961864,0.938776,0.977143,0.958333,0.966102,0.948454,0.920049,0.920167
